# Model Fitting

In [73]:
import matplotlib as m
import sqlite3
import pandas as pd
import numpy

### Initialization of variables and arrays, p_i are the probabilities, mu_i are the service times


In [74]:
p1_ = [0.34,0.60,0.25,0.15]
p2_ = [0.33,0.25,0.60,0.25]
p_ = numpy.array([p1_,p2_]);
mu_proc_ = [0.01, 0.01 ,0.01, 0.01, 0.1, 0.1, 0.1, 0.1] 
mu_disk_ = [0.1 , 1.0  ,0.1 , 1.0 , 0.1, 1.0, 0.1, 1.0]
mu_remote_ = [0.1 , 1.0  ,1.0 , 0.1 , 0.1, 1.0, 1.0, 0.1]
M = 3;

### Pre initialization of output matrix

In [75]:
throughput= numpy.zeros((4,8,49))
responseTime = numpy.zeros((4,8,49))
U = numpy.zeros((4,8,49,3))

### For each scenario, for each configuration and for each number of transactions, (clients)

- Initialize the scenario with the corresponding probabilities (p1 exit, p2 send to disk, 1-p1-p2 send to remote server)

- Initialize service times for the given configuration

- Compute the solution (e_) for the system of equations lambda x PI' = lambda

- Compute rho for each SC

- Run Buzen's algorithm and compute Utilization of each SC, throughput, responseTime


In [78]:
for scenario in range(0,4):
    scenario = 0
    p1 = p_[0][scenario]
    p2 = p_[1][scenario]    
    for config in range(0,8):
        config = 0           
        mu_proc = mu_proc_[config]
        mu_disk = mu_disk_[config]
        mu_remote = mu_remote_[config]   
        for jobs in range (0,50):
            jobs = 49
            mu = [mu_proc , mu_disk , mu_remote]
            K = jobs;
            e_proc = 1 / mu_proc
            e_disk = p2 * (e_proc)
            e_remote = (1-(p1+p2))*(e_proc)
            rho_proc = e_proc/(1/mu_proc)
            rho_disk = e_disk/(1/mu_disk)
            rho_remote = e_remote/(1/mu_remote)
            rho = [rho_proc, rho_disk, rho_remote]
            #Algoritmo di Convoluzione di Buzen

            G = numpy.ones((M,K+1))

            for i in range (1,M):
                for j in range(1,K+1):
                    G[i][j] = G[i-1][j] + rho[i]*G[i][j-1]

            E_N = K
            g = G[M-1][K-1]/G[M-1][K]
            
            U[scenario][config][jobs-1] = numpy.multiply(rho,g)

            inverse_mu = numpy.divide(1,mu)
            
            gamma = numpy.multiply(U[scenario][config][jobs-1], inverse_mu )

            throughput[scenario][config][jobs-1] = p1*gamma[0]
            responseTime[scenario][config][jobs-1] = E_N / throughput[scenario][config][jobs-1]

Now that all the computation's are done, we can compare our QT results with the simulation's results